In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
spotify_tracks = pd.read_csv('archive/tracks.csv')
print(spotify_tracks.shape)
spotify_tracks.head()

(586672, 20)


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [3]:
clean_df = spotify_tracks.copy()
print('before',clean_df.shape)

# remove nonlatin chars
clean_df['name'] = clean_df['name'].str.encode('ascii', 'ignore').str.decode('ascii')

# remove empty rows
clean_df = clean_df[clean_df["name"] != " "]
clean_df = clean_df[clean_df["name"] != ""]

print('after', clean_df.shape)

before (586672, 20)
after (547069, 20)


In [4]:
clean_df['release_date']= pd.to_datetime(clean_df['release_date'])
clean_df = clean_df[clean_df['release_date'].dt.year >= 2019]

In [5]:
clean_df = clean_df[['id','name','artists']]
clean_df

,id,name,artists
39511,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,['The Toys']
39581,55qyghODi24yaDgKBI6lx0,"The Circle Game - Live at The 2nd Fret, Philad...",['Joni Mitchell']
39583,00xemFYjQNRpOlPhVaLAHa,"Urge For Going - Live at The 2nd Fret, Philade...",['Joni Mitchell']
39585,26g4FBGTB9YEj7q4HlblFf,"Brandy Eyes - Live at The 2nd Fret, Philadelph...",['Joni Mitchell']
39587,2lm5FQJRHvc3rUN5YHpEWj,What's The Story Mr. Blue - Live at The 2nd Fr...,['Joni Mitchell']
...,...,...,...
586665,0SjsIzJkZfDU7wlcdklEFR,John Brown's Song,['Gregory Oberle']
586666,1ZwZsVZUiyFwIHMNpI3ERt,Skyscraper,['Emilie Chin']
586668,0NuWgxEp51CutD2pJoF4OM,blind,['ROLE MODEL']
586669,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,['FINNEAS']


In [6]:
# keep only first 1000 for test
test = clean_df.tail(10)


In [8]:
from lyricsgenius import Genius
import re
import nltk 
from nltk import pos_tag
#nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer


In [9]:
client_id = 'BMU2d7PVnIvEnVFKQiMlwJFcSffuHq2epuHYfqTstC7CiqLqSC42KMDTEVTPnRsY'
client_secret ='6Sh1I8isQGva9bIDg8qDnCSEnHbfQy71SurcywPLj2s8R9S1W8MdpprpYdqSPR2AO8c79njZlhSl9RHnxB_5vw'
token = 'sL8fGLHNCUnND1eoeMd6t3-gLWvSet8sU-zoBfauaIPuOXlbKuZrdX2hJiXGFWjD'

def get_lyrics(artist, song_title):

    genius = Genius(token)
    genius.remove_section_headers = True

    try:
        lyrics = genius.search_song(song_title, artist).lyrics
    except:
        lyrics=''

    lyrics = clean_lyrics(lyrics)
    return lyrics

#Mapping POS tag to first character lemmatize() accepts
def get_wordnet_pos(word):
        
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

#Function to Lemmatize every word and remove stopwords 
def lemma(text):
    
    # Setting stopwords
    stop = set(stopwords.words("english"))

    #Initializing Lemmatizer
    lemmatizer = WordNetLemmatizer()

    text = [lemmatizer.lemmatize(x, get_wordnet_pos(x)) for x in nltk.word_tokenize(text)]
    text = [x for x in text if x not in stop]
    return ' '.join(text)

def clean_lyrics(lyrics):

    lyrics = " ".join(re.findall("[a-zA-Z]+", lyrics))
    lyrics = lyrics.lower()

    #lyrics = lemma(lyrics)

    return lyrics

In [10]:
test

,id,name,artists
586660,4SMp3hkQV64IoaaKsBQoBo,Edelweiss,['Ludovico Sagese']
586661,27kcZEJvhkb1rzZS9gCpdA,remember the mornings,['Clinton Kane']
586662,4Zp3rm12p5PiHToYJflmyy,Meet Again,['KIMSEJEONG']
586663,4ow9HehIdFii1cggylW2k0,- DJ,"['程響', '阿卓']"
586664,1Kzjk1EyngBcP4T8x3fyqv,(...),['Boon Hui Lu']
586665,0SjsIzJkZfDU7wlcdklEFR,John Brown's Song,['Gregory Oberle']
586666,1ZwZsVZUiyFwIHMNpI3ERt,Skyscraper,['Emilie Chin']
586668,0NuWgxEp51CutD2pJoF4OM,blind,['ROLE MODEL']
586669,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,['FINNEAS']
586670,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,"['Gentle Bones', 'Clara Benin']"


In [11]:
lyrics_list = []
for index, row in test.iterrows():
    res = row['artists'].strip('][').strip('\'').split(', ')
    print(row['name'])
    art = res[0].replace('\'', '')
    try:
        lyrics_list.append(get_lyrics(row['name'], art))
    except:
        lyrics_list.append("")

test['lyrics']=lyrics_list

Edelweiss
Searching for "Ludovico Sagese" by Edelweiss...
No results found for: 'Ludovico Sagese Edelweiss'
remember the mornings
Searching for "Clinton Kane" by remember the mornings...
Done.
Meet Again
Searching for "KIMSEJEONG" by Meet Again...
No results found for: 'KIMSEJEONG Meet Again'
 - DJ
Searching for "程響" by  - DJ...
Done.
 (...)
Searching for "Boon Hui Lu" by  (...)...
Done.
John Brown's Song
Searching for "Gregory Oberle" by John Brown's Song...
No results found for: 'Gregory Oberle John Brown's Song'
Skyscraper
Searching for "Emilie Chin" by Skyscraper...
Done.
blind
Searching for "ROLE MODEL" by blind...
Done.
What They'll Say About Us
Searching for "FINNEAS" by What They'll Say About Us...
Done.
A Day At A Time
Searching for "Gentle Bones" by A Day At A Time...
Done.


/var/folders/25/15h8p44s3_z09_s99_nxm3480000gn/T/ipykernel_45162/2628125879.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['lyrics']=lyrics_list


In [12]:
test

,id,name,artists,lyrics
586660,4SMp3hkQV64IoaaKsBQoBo,Edelweiss,['Ludovico Sagese'],
586661,27kcZEJvhkb1rzZS9gCpdA,remember the mornings,['Clinton Kane'],contributors remember the mornings lyrics i ve...
586662,4Zp3rm12p5PiHToYJflmyy,Meet Again,['KIMSEJEONG'],
586663,4ow9HehIdFii1cggylW2k0,- DJ,"['程響', '阿卓']",contributorbe mine lyricsmove it to the left m...
586664,1Kzjk1EyngBcP4T8x3fyqv,(...),['Boon Hui Lu'],contributormessed up lyrics give it up give it...
586665,0SjsIzJkZfDU7wlcdklEFR,John Brown's Song,['Gregory Oberle'],
586666,1ZwZsVZUiyFwIHMNpI3ERt,Skyscraper,['Emilie Chin'],contributorsscarface script lyricsscarface by ...
586668,0NuWgxEp51CutD2pJoF4OM,blind,['ROLE MODEL'],contributors blind lyrics she got the gold hoo...
586669,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,['FINNEAS'],contributorstranslations what they ll say abou...
586670,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,"['Gentle Bones', 'Clara Benin']",contributorsa day at a time lyrics people all ...


To do:
- remove genius api addons